# SI 618 F22 Lab 6

### 0. Setup
The output of input_file.take(5) should look exactly like this

` 
["I'Alg\t1850\t1\t1",
 "I'Alg\t1855\t2\t2",
 "I'Alg\t1883\t1\t1",
 "I'Alg\t1886\t3\t3",
 "I'Alg\t1939\t1\t1"]
`

In [1]:
# Import the necessary Spark library classes
from pyspark import SparkConf, SparkContext

# Grab the input and output path
input_path = '/scratch/siads618f22_class_root/siads618f22_class/shared_data/lab6_data'
output_path = './ngram_out/'

# Create a configuration for this Spark job
conf = SparkConf().setAppName('AnnualWordLength').set("spark.hadoop.validateOutputSpecs", "false")

# Create a context for the job. The context is used to manage the job at a
# high level.
sc = SparkContext.getOrCreate(conf=conf)

In [2]:
input_file = sc.textFile(input_path)

In [3]:
input_file.take(5)

["I'Alg\t1850\t1\t1",
 "I'Alg\t1855\t2\t2",
 "I'Alg\t1883\t1\t1",
 "I'Alg\t1886\t3\t3",
 "I'Alg\t1896\t2\t2"]

### 1.	Transform each line of the original dataset into an array of the form [1gram, year, occurrences, volumes]. Store the transformed dataset in "data". (Hint: use map transformation)
The output of data.take(5) should look exactly like this

`
[["I'Alg", '1850', '1', '1'],
 ["I'Alg", '1855', '2', '2'],
 ["I'Alg", '1883', '1', '1'],
 ["I'Alg", '1886', '3', '3'],
 ["I'Alg", '1896', '2', '2'],]`

In [4]:
data = input_file.map(lambda line: line.split('\t'))

In [5]:
data.take(5)

[["I'Alg", '1850', '1', '1'],
 ["I'Alg", '1855', '2', '2'],
 ["I'Alg", '1883', '1', '1'],
 ["I'Alg", '1886', '3', '3'],
 ["I'Alg", '1896', '2', '2']]

### 2.	Create a new dataset "length" with each row containing an array of the form [year, sum_of_the_length_of_all_words_in_that_year]. 
Hint: A word could occur multiple times in a year, and you will need to count that word multiple times when calculating the total length.

Remember to convert the string values to integer in the map function. 

The output of length.take(5) should look like this

`
[(1938, 2021053404),
 (1836, 551487404),
 (1734, 13722773),
 (1632, 57106),
 (1837, 508248822),]
 `

In [6]:
# map each row to a (year, sum of length of one word in that year) pair
lengthAll = data.map(lambda info: (int(info[1]), int(info[2])*len(info[0])))
# reduce lengthAll to length
length = lengthAll.reduceByKey(lambda a,b: a+b)

In [10]:
length.take(5)

[(1938, 2021053404),
 (1836, 551487404),
 (1734, 13722773),
 (1632, 57106),
 (1837, 508248822)]

### 3.	Create a new dataset "count" with each row containing an array of the form [year, total_number of words]
The output of count.take(5) should look like this

`
[(1938, 288113442.0),
 (1836, 80514369.0),
 (1734, 2123777.0),
 (1632, 8382.0),
 (1939, 285347395.0),]
 `

In [11]:
countAll = data.map(lambda info: (int(info[1]), float(info[2])))
count = countAll.reduceByKey(lambda a,b: a+b)

In [16]:
count.take(5)

[(1938, 288113442.0),
 (1836, 80514369.0),
 (1734, 2123777.0),
 (1632, 8382.0),
 (1939, 285347395.0)]

### 4.	Create a new dataset "avgLength" with each row containing an array of the form [year, average word length] by dividing length by words. (Hint: join the length and count first and use map to obtain the new dataset)

The output of avgLength.take(5) should look like this

`
[(1938, 7.014783447694884),
 (1836, 6.84955258110512),
 (1734, 6.461494309430792),
 (1632, 6.812932474349797),
 (1939, 7.041908533981886),]
 `

In [17]:
avgLength = length.join(count).map(lambda lc_info: (lc_info[0], lc_info[1][0]/lc_info[1][1]))

In [18]:
avgLength.take(5)

[(1938, 7.014783447694884),
 (1836, 6.84955258110512),
 (1734, 6.461494309430792),
 (1632, 6.812932474349797),
 (1939, 7.041908533981886)]

### 5. Sort the results by year in chronological order

The output of avgLengthSorted.take(5) should look like this

`
[(1505, 6.618036653964798),
 (1507, 6.304188864514781),
 (1515, 7.488374666097649),
 (1520, 6.245398773006135),
 (1524, 7.282010767790262)]
 `

In [19]:
avgLengthSorted = avgLength.sortBy(lambda a_info: a_info[0], ascending = True)
avgLengthSorted.take(5)

[(1505, 6.618036653964798),
 (1507, 6.304188864514781),
 (1515, 7.488374666097649),
 (1520, 6.245398773006135),
 (1524, 7.282010767790262)]

### 6. Save the results in the specified output directory.

In [20]:
avgLengthSorted.saveAsTextFile(output_path)

In [21]:
# Finally, let Spark know that the job is done.
sc.stop()

In [22]:
# concatenate the results to one file
!cat ./ngram_out/part* >  sijuntao_ngrams_output.csv